<a href="https://colab.research.google.com/github/AhmadArrabi/Cat-Generation-using-GANs/blob/main/FCC_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy
from PIL import Image
import tensorflow as tf
from tqdm import tqdm
from tensorflow import keras

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data/dataset/dataset3/cats2",
    label_mode=None,
    image_size=(64, 64),
    batch_size=128,
    color_mode = 'rgb'
    )

In [ ]:
dataset = dataset.map(lambda x: x / 255.)

In [2]:
discriminator = keras.Sequential(
    [
     keras.Input(shape = (64,64,3)),
     keras.layers.Conv2D(filters=128, kernel_size=5, strides=2, padding='same'),
     #keras.layers.AveragePooling2D(pool_size=(2,2)),
     keras.layers.BatchNormalization(),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2D(filters=256, kernel_size=5, strides=2, padding='same'),
     #keras.layers.AveragePooling2D(pool_size=(2,2)),
     keras.layers.BatchNormalization(),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2D(filters=512, kernel_size=5, strides=2, padding='same'),
     #keras.layers.AveragePooling2D(pool_size=(2,2)),
     keras.layers.BatchNormalization(),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Conv2D(filters=1024, kernel_size=5, strides=2, padding='same'),
     #keras.layers.AveragePooling2D(pool_size=(2,2)),
     keras.layers.BatchNormalization(),
     keras.layers.LeakyReLU(0.2),
     keras.layers.Flatten(),
     #keras.layers.Dense(1024),
     #keras.layers.LeakyReLU(0.2),
     keras.layers.Dense(512),
     keras.layers.LeakyReLU(0.2),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(128),
     keras.layers.LeakyReLU(0.2),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(16),
     keras.layers.LeakyReLU(0.2),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(1, activation = 'sigmoid')
    ]
)
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       9728      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 512)         3

In [3]:
latent_dim = 128

generator = keras.Sequential(
    [
     keras.Input(shape = (latent_dim,)),
     keras.layers.Dense(64, activation='relu'),
     keras.layers.BatchNormalization(),
     keras.layers.Dense(512, activation='relu'),
     keras.layers.BatchNormalization(),
     #keras.layers.Dense(1024, activation='relu'),
     keras.layers.Dense(16384, activation='relu'),
     keras.layers.BatchNormalization(),
     keras.layers.Reshape((4,4,1024)),
     keras.layers.Conv2DTranspose(filters=512, kernel_size=4, strides=2, padding='same', activation='relu'),
     keras.layers.BatchNormalization(),
     keras.layers.Conv2DTranspose(filters=256, kernel_size=4, strides=2, padding='same', activation='relu'),
     keras.layers.BatchNormalization(),
     keras.layers.Conv2DTranspose(filters=128, kernel_size=4, strides=2, padding='same', activation='relu'),
     keras.layers.BatchNormalization(),
     keras.layers.Conv2DTranspose(filters=3, kernel_size=4, strides=2, padding='same', activation='tanh'),
     #keras.layers.BatchNormalization(),
    ]
)
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization_7 (Batch (None, 64)                256       
_________________________________________________________________
dense_5 (Dense)              (None, 512)               33280     
_________________________________________________________________
batch_normalization_8 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 16384)             8404992   
_________________________________________________________________
batch_normalization_9 (Batch (None, 16384)             65536     
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 1024)       

In [ ]:
gen_opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.00001)
disc_opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.00001)

In [ ]:
loss = keras.losses.BinaryCrossentropy()
latent_dim = 128

In [ ]:
import tensorflow as tf
from tqdm import tqdm
from tensorflow import keras

with tf.device('/device:GPU:0'):
  for epoch in range(100):
    for idx, real in enumerate(tqdm(dataset)):
      batch_size = real.shape[0]
      random_latent_vector = tf.random.normal(shape=(batch_size, latent_dim))
      fake = generator(random_latent_vector)

      if idx % 100 == 0:
        img = keras.preprocessing.image.array_to_img(fake[0])
        img.save("/content/drive/MyDrive/data/dataset/generated4/take6/generated_img_%03d_%d.jpg" % (epoch, idx))
      
      #Train Discriminator ylog(y') + (1-y)log(1-y') => ylog(D(x)) + (1-y)log(1-D(G(z)))
      with tf.GradientTape() as disc_tape:
        loss_disc_real = loss(tf.ones((batch_size,1)), discriminator(real))
        loss_disc_fake = loss(tf.zeros((batch_size,1)), discriminator(fake))
        loss_disc = (loss_disc_fake + loss_disc_real)
      
      grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
      disc_opt.apply_gradients(
          zip(grads, discriminator.trainable_weights)
      )
      
      #Train generator: log(D(G(z)))
      with tf.GradientTape() as gen_tape:
        fake = generator(random_latent_vector)
        output = discriminator(fake)
        loss_gen = loss(tf.ones(batch_size,1), output)
      
      grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
      gen_opt.apply_gradients(
          zip(grads, generator.trainable_weights)
      )
      #2h+ without pooling layers

 33%|███▎      | 12/36 [00:22<00:44,  1.83s/it]


KeyboardInterrupt: ignored

## Save

In [ ]:
import os.path
if os.path.isfile('/content/drive/MyDrive/data/models/FCC-GAN/generator.h5') is False:
  generator.save('/content/drive/MyDrive/data/models/FCC-GAN/generator.h5')

In [ ]:
if os.path.isfile('/content/drive/MyDrive/data/models/FCC-GAN/dicriminator.h5') is False:
  discriminator.save('/content/drive/MyDrive/data/models/FCC-GAN/dicriminator.h5')

In [ ]:
#for epoch in range(100):
#  for idx, real in enumerate(tqdm(dataset)):
#    batch_size = real.shape[0]
#
#    #generate random images
#    random_latent_vector = tf.random.normal(shape=(batch_size, latent_dim))
#    fake = generator(random_latent_vector)
#    
#    #save image in each epoch
#    if idx % 100 == 0:
#        img = keras.preprocessing.image.array_to_img(fake[0])
#        img.save("/content/drive/MyDrive/data/dataset/generated4/generated_img_%03d_%d.jpg" % (epoch, idx))
#    
#    #train discriminator
#    with tf.GradientTape() as disc_tape:
#      